In [1]:
!pip install --user tensorflow_text

     |████████████████████████████████| 3.0MB 3.4MB/s eta 0:00:01
     |████████████████████████████████| 516.2MB 20kB/s s eta 0:00:01     |██████████████████████████▍     | 425.2MB 46.7MB/s eta 0:00:02
     |████████████████████████████████| 460kB 40.8MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 41.5MB/s eta 0:00:01
     |████████████████████████████████| 3.0MB 32.0MB/s eta 0:00:01
     |████████████████████████████████| 26.1MB 41.8MB/s eta 0:00:01
     |████████████████████████████████| 778kB 25.8MB/s eta 0:00:01
ERROR: allennlp 0.9.0 requires flaky, which is not installed.
ERROR: allennlp 0.9.0 requires responses>=0.7, which is not installed.
ERROR: tpot 0.11.1 has requirement scikit-learn>=0.22.0, but you'll have scikit-learn 0.21.3 which is incompatible.
ERROR: optuna 1.0.0 has requirement scipy<1.4.0, but you'll have scipy 1.4.1 which is incompatible.
ERROR: kmeans-smote 0.1.2 has requirement imbalanced-learn<0.5,>=0.4.0, but you'll have imbalanced-learn 0.6.1 w

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [2]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score
from sklearn.utils import shuffle
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text

from tqdm import tqdm
import numpy as np
import pandas as pd
import re

## Loadind datasets.

In [3]:
train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")
submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

## Data cleaning.

In [4]:
def clean(text):
    text = re.sub(r"http\S+", " ", text) # remove urls
    text = re.sub(r"RT ", " ", text) # remove rt
    text = re.sub(r"[^a-zA-Z\'\.\,\d\s]", " ", text) # remove special character except # @ . ,
    text = re.sub(r"[0-9]", " ", text) # remove number
    text = re.sub(r'\t', ' ', text) # remove tabs
    text = re.sub(r'\n', ' ', text) # remove line jump
    text = re.sub(r"\s+", " ", text) # remove extra white space
    text = text.strip()
    return text

In [5]:
train.text = train.text.apply(clean)
test.text = test.text.apply(clean)

## Loading universal sentences encoder.

In [6]:
use = hub.load("https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3")

## Sentences embedding.

In [7]:
X_train = []
for r in tqdm(train.text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_train.append(review_emb)

X_train = np.array(X_train)
y_train = train.target.values

X_test = []
for r in tqdm(test.text.values):
  emb = use(r)
  review_emb = tf.reshape(emb, [-1]).numpy()
  X_test.append(review_emb)

X_test = np.array(X_test)

100%|██████████| 3263/3263 [02:06<00:00, 25.82it/s]


## Sampling over data train.

In [8]:
train_arrays, test_arrays, train_labels, test_labels = train_test_split(X_train,y_train,test_size=0.05)

## Training svm model.

In [9]:
def svc_param_selection(X, y, nfolds):
    #Cs = [0.001, 0.01, 0.1, 1, 10]
    Cs = [1.070, 1.074, 1.075, 1.1, 1.125]
    #gammas = [0.001, 0.01, 0.1, 1]
    gammas = [2.065,2.075, 2.08]
    param_grid = {'C': Cs, 'gamma' : gammas}
    grid_search = GridSearchCV(SVC(kernel='rbf'), param_grid, cv=nfolds, n_jobs=8)
    grid_search.fit(X, y)
    grid_search.best_params_
    return grid_search

model = svc_param_selection(train_arrays,train_labels, 5)

## Best parameters:

In [10]:
model.best_params_

{'C': 1.07, 'gamma': 2.075}

## Predictions over valuation data.

In [11]:
pred = model.predict(test_arrays)

## Accuracy

In [12]:
cm = confusion_matrix(test_labels,pred)
cm

array([[208,  12],
       [ 39, 122]])

In [13]:
accuracy = accuracy_score(test_labels,pred)
accuracy

0.8661417322834646

## Making submission

In [17]:
test_pred = model.predict(X_test)
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)